# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos Semanas 7 y 8 : LDA y LMM audio-a-texto**

* **Nombres y matrículas:**

  *   Elemento de lista
  *   Elemento de lista
  *   Elemento de lista

* **Número de Equipo:**


* ##### **En cada ejercicio pueden importar los paquetes o librerías que requieran.**

* ##### **En cada ejercicio pueden incluir las celdas y líneas de código que deseen.**

In [14]:
import os
import requests
import json
import openai
from dotenv import load_dotenv
import re
from nltk.tokenize import word_tokenize
import nltk

# **Ejercicio 1:**

* #### **Liga de los audios de las fábulas de Esopo:** https://www.gutenberg.org/ebooks/21144

* #### **Descargar los 10 archivos de audio solicitados: 1, 4, 5, 6, 14, 22, 24, 25, 26, 27.**



In [15]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...

# Base URL and target folder
base_url = "https://www.gutenberg.org/files/21144/mp3/21144-{:02d}.mp3"
target_folder = "downloads"
file_numbers = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Create target folder if it doesn't exist
os.makedirs(target_folder, exist_ok=True)

# Download loop
for num in file_numbers:
    file_url = base_url.format(num)
    filename = f"21144-{num:02d}.mp3"
    file_path = os.path.join(target_folder, filename)

    if os.path.exists(file_path):
        print(f"Already exists: {filename}")
        continue

    print(f"Downloading: {filename}")
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {filename}")
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {filename}: {e}")


Already exists: 21144-01.mp3
Already exists: 21144-04.mp3
Already exists: 21144-05.mp3
Already exists: 21144-06.mp3
Already exists: 21144-14.mp3
Already exists: 21144-22.mp3
Already exists: 21144-24.mp3
Already exists: 21144-25.mp3
Already exists: 21144-26.mp3
Already exists: 21144-27.mp3


# **Ejercicio 2a:**

* #### **Comenten el por qué del modelo seleccionado para extracción del texto de los audios.**

* #### **Extraer el contenido de los audios en texto.**

* #### **Sugerencia:** pueden extraerlo en un formato de diccionario, clave:valor $→$ {audio01:fabula01, ...}

In [16]:
# Load environment variables from a `.env` file (e.g., OPENAI_API_KEY=sk-...)
load_dotenv()

# Create the OpenAI client using the API key loaded from environment
client = openai.OpenAI()

# Folder where the MP3 files are stored
AUDIO_FOLDER = "downloads"

# Path to the cache file that stores transcripts to avoid reprocessing
CACHE_FILE = "transcripts_api.json"

# List of file numbers we want to transcribe (e.g., 21144-01.mp3, 21144-04.mp3, etc.)
FILE_NUMBERS = [1, 4, 5, 6, 14, 22, 24, 25, 26, 27]

# Load the transcript cache if it already exists, otherwise start with an empty dict
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, "r", encoding="utf-8") as f:
        transcript_cache = json.load(f)
else:
    transcript_cache = {}

# Function to transcribe a single MP3 file using the OpenAI Whisper-1 API
def transcribe_with_openai(mp3_path):
    # If the transcription is already cached, return it to avoid extra API cost
    if mp3_path in transcript_cache:
        print(f"Transcript found in cache: {mp3_path}")
        return transcript_cache[mp3_path]

    print(f"Transcribing with Whisper-1: {mp3_path}")
    try:
        # Open the audio file in binary mode
        with open(mp3_path, "rb") as audio_file:
            # Send the file to OpenAI's Whisper-1 transcription API
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
            # Extract the transcript text from the API response
            text = transcript.text

            # Save the result in the cache
            transcript_cache[mp3_path] = text

            # Write the updated cache back to the JSON file
            with open(CACHE_FILE, "w", encoding="utf-8") as f:
                json.dump(transcript_cache, f, ensure_ascii=False, indent=2)

            return text
    except Exception as e:
        print(f"Error transcribing {mp3_path}: {e}")
        return None

# Main loop: go through each file number and transcribe the corresponding MP3
for num in FILE_NUMBERS:
    file_name = f"21144-{num:02d}.mp3"  # Format file name with leading zero (e.g., 01, 04, etc.)
    file_path = os.path.join(AUDIO_FOLDER, file_name)  # Full path to the file

    # Skip files that don't exist
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    # Transcribe the file and print a short summary
    result_text = transcribe_with_openai(file_path)
    if result_text:
        print(f"Transcribed ({file_name}): {len(result_text.split())} palabras")


Transcript found in cache: downloads/21144-01.mp3
Transcribed (21144-01.mp3): 103 palabras
Transcript found in cache: downloads/21144-04.mp3
Transcribed (21144-04.mp3): 142 palabras
Transcript found in cache: downloads/21144-05.mp3
Transcribed (21144-05.mp3): 135 palabras
Transcript found in cache: downloads/21144-06.mp3
Transcribed (21144-06.mp3): 152 palabras
Transcript found in cache: downloads/21144-14.mp3
Transcribed (21144-14.mp3): 103 palabras
Transcript found in cache: downloads/21144-22.mp3
Transcribed (21144-22.mp3): 103 palabras
Transcript found in cache: downloads/21144-24.mp3
Transcribed (21144-24.mp3): 140 palabras
Transcript found in cache: downloads/21144-25.mp3
Transcribed (21144-25.mp3): 92 palabras
Transcript found in cache: downloads/21144-26.mp3
Transcribed (21144-26.mp3): 118 palabras
Transcript found in cache: downloads/21144-27.mp3
Transcribed (21144-27.mp3): 95 palabras


In [17]:
transcript_cache

{'downloads/21144-01.mp3': 'Las fábulas de Sopo Grabado para LibriVox.org por Paulino www.paulino.info Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor. Fin de la fábula Esta es una grabación del dominio público.',
 'downloads/21144-04.mp3': 'Las fábulas de Esopo, grabado para LibriVox.org por Roberto Antonio Muñoz, fábula número 64, El Lobo y la Cruz. A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correra a una grulla y le pidió que le salvara de aquella situación y que enseguida le pagaría por ello. Aceptó la g

## Justificación técnica de **whisper-1**
| Criterio                          | Valoración del modelo `whisper-1` de OpenAI                                                                                                                               |
| --------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Precisión en español**          | Entrenado multilingüe sobre millones de horas, tiene una precisión excelente en español, comparable o superior a modelos fine-tuneados.                                   |
| **Robustez ante ruidos**          | Capaz de manejar variaciones en calidad de audio, acentos, pausas, y entonación sin afectar notablemente la transcripción.                                                |
| **Preprocesamiento mínimo**       | Genera texto limpio, con puntuación adecuada y frases coherentes, lo que reduce la necesidad de procesamiento adicional.                                                  |
| **Uso sin infraestructura local** | No requiere GPU ni instalación de modelos grandes; todo el procesamiento ocurre en la nube de OpenAI. Ideal para laptops o estaciones sin capacidad de cómputo intensiva. |
| **Velocidad y escalabilidad**     | Muy rápido y escalable; ideal para proyectos pequeños y también para futuras automatizaciones más amplias.                                                                |
| **Facilidad de integración**      | Uso vía API REST con librerías bien documentadas (`openai-python`), lo que simplifica la integración en pipelines.                                                        |


## Comparación con modelos alternativos
- **Modelos open-source (clu-ling/whisper-large-v2-spanish):** aunque preciso, requieren gran cantidad de RAM y preferentemente GPU para un rendimiento adecuado. En sistemas solo con CPU, el tiempo de procesamiento puede ser excesivo.

- **Modelos pequeños (whisper-small, medium):** menos exigentes, pero pierden precisión, especialmente en español y en audios narrativos donde cada palabra tiene peso semántico.

## Conclusión
**whisper-1** es el modelo más adecuado para el caso de uso porque:

- Tiene la mayor precisión posible en español sin requerir entrenamiento adicional.

- Funciona perfectamente sin necesidad de GPU, ideal para un entorno local.

- Genera transcripciones limpias, coherentes y con puntuación.

- Minimiza la necesidad de procesamiento posterior (limpieza, normalización, etc.).

- Evita complicaciones de instalación, consumo de recursos y errores locales.

# **Ejercicio 2b:**

* #### **Eliminar el inicio y final comunes de los textos extraídos de cada fábula.**

* #### **Sugerencia:** Pueden guardar esta información en un archivo tipo JSON, para que al estar probando diferentes opciones en los ejercicios siguientes, puedan recuperar rápidamente la información de cada video/fábula.

In [18]:
# Load original transcripts
with open("transcripts_api.json", "r", encoding="utf-8") as f:
    transcripts = json.load(f)

cleaned_transcripts = {}

for path, text in transcripts.items():
    original_text = text.strip()

    # Step 1: Start from "Fábula número X"
    match = re.search(r"(F[áa]bula número \d+)", original_text, flags=re.IGNORECASE)
    if match:
        cleaned = original_text[match.start():]
    else:
        cleaned = original_text  # fallback if pattern not found

    # Step 2: Remove from "Fin de la fábula" OR "Fin de fábula" (inclusive)
    outro_match = re.search(r"Fin de (la )?fábula", cleaned, flags=re.IGNORECASE)
    if outro_match:
        cleaned = cleaned[:outro_match.start()].strip()

    cleaned_transcripts[path] = cleaned

# Save to cleaned JSON file
with open("transcripts_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_transcripts, f, ensure_ascii=False, indent=2)

print("Limpieza completada. Archivo guardado como 'transcripts_cleaned.json'")


Limpieza completada. Archivo guardado como 'transcripts_cleaned.json'


In [19]:
cleaned_transcripts

{'downloads/21144-01.mp3': 'Fábula número 61 El lobo y el cordero en el templo Dándose cuenta de que era perseguido por un lobo, un pequeño corderito decidió refugiarse en un templo cercano. Lo llamó lobo y le dijo que si el sacrificador lo encontraba allí adentro, lo inmolaría a su dios. Mejor así, replicó el cordero, prefiero ser víctima para un dios a tener que perecer en tus colmillos. Si sin remedio vamos a ser sacrificados, más nos vale que sea con el mayor honor.',
 'downloads/21144-04.mp3': 'fábula número 64, El Lobo y la Cruz. A un lobo que comía un hueso, se le atragantó el hueso en la garganta y corría por todas partes en busca de auxilio. Encontró en su correra a una grulla y le pidió que le salvara de aquella situación y que enseguida le pagaría por ello. Aceptó la grulla e introdujo su cabeza en la boca del lobo, sacando de la garganta el hueso atravesado. Pidió entonces la cancelación de la paga convenida. Oye, amiga, dijo el lobo, ¿no crees que es suficiente paga con ha

# **Ejercicio 3:**

* #### **Apliquen el proceso de limpieza que consideren adecuado.**

* #### **Justifiquen los pasos de limpieza utilizados. Tomen en cuenta que el texto extraído de cada fábula es relativamente pequeño.**

* #### **En caso de que decidan no aplicar esta etapa de limpieza, deberán justificarlo.**

In [20]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...
# Download tokenizer (if needed)
nltk.download("punkt_tab")

# Load cleaned transcripts
with open("transcripts_cleaned.json", "r", encoding="utf-8") as f:
    cleaned = json.load(f)

tokenized = {}

for path, text in cleaned.items():
    # Lowercase the text
    text = text.lower()

    # Remove punctuation (keep accented letters and ñ)
    text = re.sub(r"[^\w\sáéíóúüñ]", "", text)

    # Tokenize
    tokens = word_tokenize(text, language="spanish")

    tokenized[path] = tokens

# Save tokenized result
with open("transcripts_tokenized.json", "w", encoding="utf-8") as f:
    json.dump(tokenized, f, ensure_ascii=False, indent=2)

print("Tokenización completada. Guardado en 'transcripts_tokenized.json'")

Tokenización completada. Guardado en 'transcripts_tokenized.json'


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/eahumada/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [21]:
tokenized

{'downloads/21144-01.mp3': ['fábula',
  'número',
  '61',
  'el',
  'lobo',
  'y',
  'el',
  'cordero',
  'en',
  'el',
  'templo',
  'dándose',
  'cuenta',
  'de',
  'que',
  'era',
  'perseguido',
  'por',
  'un',
  'lobo',
  'un',
  'pequeño',
  'corderito',
  'decidió',
  'refugiarse',
  'en',
  'un',
  'templo',
  'cercano',
  'lo',
  'llamó',
  'lobo',
  'y',
  'le',
  'dijo',
  'que',
  'si',
  'el',
  'sacrificador',
  'lo',
  'encontraba',
  'allí',
  'adentro',
  'lo',
  'inmolaría',
  'a',
  'su',
  'dios',
  'mejor',
  'así',
  'replicó',
  'el',
  'cordero',
  'prefiero',
  'ser',
  'víctima',
  'para',
  'un',
  'dios',
  'a',
  'tener',
  'que',
  'perecer',
  'en',
  'tus',
  'colmillos',
  'si',
  'sin',
  'remedio',
  'vamos',
  'a',
  'ser',
  'sacrificados',
  'más',
  'nos',
  'vale',
  'que',
  'sea',
  'con',
  'el',
  'mayor',
  'honor'],
 'downloads/21144-04.mp3': ['fábula',
  'número',
  '64',
  'el',
  'lobo',
  'y',
  'la',
  'cruz',
  'a',
  'un',
  'lobo',

# **Ejercicio 4:**

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 5a y 5b:**

* #### **5a: Mediante el LLM que hayan seleccionado, generar un único enunciado que describa o resuma cada fábula.**

* #### **5b: Mediante el LLM que hayan seleccionado, generar tres posibles enunciados diferentes relacionados con la historia de la fábula.**

* #### **Sugerencia:** En realidad los dos incisos a y b se pueden obtener con un solo prompt que solicite la información y el formato correspondiente para cada una de estas partes. Por ejemplo, para cada fábula la salida puede ser un primer enunciado genérico que resume o describe dicha temática; seguido de tres enunciados, cada uno hablando sobre una situación o parte diferente de la fábula.

In [ ]:
# Incluyan a continuación todas las celdas (de código o texto) que deseen...





# **Ejercicio 6:**

* #### **Incluyan sus conclusiones de la actividad audio-a-texto:**



None

# **Fin de la actividad LDA y LMM: audio-a-texto**